In [3]:
import pandas as pd 
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import datetime

In [11]:
df_client = pd.read_csv('./data/predict-energy-behavior-of-prosumers/client.csv')
df_train = pd.read_csv('./data/predict-energy-behavior-of-prosumers/train.csv').sample(100000, random_state=4242)
df_gas_prices = pd.read_csv('./data/predict-energy-behavior-of-prosumers/gas_prices.csv')
df_electricity = pd.read_csv('./data/predict-energy-behavior-of-prosumers/electricity_prices.csv')
df_forecast_weather = pd.read_csv('./data/predict-energy-behavior-of-prosumers/forecast_weather.csv')
df_historical_weather = pd.read_csv('./data/predict-energy-behavior-of-prosumers/historical_weather.csv')
df_weather_station = pd.read_csv('./data/predict-energy-behavior-of-prosumers/weather_station_to_county_mapping.csv')

In [4]:
df_train.sample(10000)

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
300277,0,0,1,433.396,1,2021-12-10 06:00:00,100,300277,0
1950399,13,0,3,27.155,1,2023-05-10 18:00:00,616,1950399,51
1971589,0,1,2,22.899,1,2023-05-17 07:00:00,623,1971589,61
251822,7,0,2,0.000,0,2021-11-24 05:00:00,84,251822,26
1688322,12,1,3,1.150,0,2023-02-16 23:00:00,533,1688322,49
...,...,...,...,...,...,...,...,...,...
372590,4,1,1,0.000,0,2022-01-03 02:00:00,124,372590,66
862765,0,1,1,228.459,1,2022-06-05 14:00:00,277,862765,4
1955448,15,0,3,203.256,0,2023-05-12 07:00:00,618,1955448,58
940474,15,1,3,356.604,0,2022-06-29 17:00:00,301,940474,60


In [12]:
df_electricity["origin_date"].nunique()

15286

In [13]:
df_electricity["date"] = pd.to_datetime(df_electricity["forecast_date"])
df_electricity["date"] = df_electricity["date"].apply(lambda x : x.strftime("%d-%m-%y"))
df_electricity["date"]

0        01-09-21
1        01-09-21
2        01-09-21
3        01-09-21
4        01-09-21
           ...   
15281    30-05-23
15282    30-05-23
15283    30-05-23
15284    30-05-23
15285    30-05-23
Name: date, Length: 15286, dtype: object

In [14]:
df_client["date_str"] = df_client["date"].apply(lambda x : str(x))
df_electricity["date_str"] = df_electricity["date"].apply(lambda x : str(x))

Feature Engineering

In [15]:
df_electricity

,forecast_date,euros_per_mwh,origin_date,data_block_id,date,date_str
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1,01-09-21,01-09-21
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1,01-09-21,01-09-21
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1,01-09-21,01-09-21
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1,01-09-21,01-09-21
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1,01-09-21,01-09-21
...,...,...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637,30-05-23,30-05-23
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637,30-05-23,30-05-23
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637,30-05-23,30-05-23
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637,30-05-23,30-05-23


Creation Colonne 

In [16]:
df_train["price_elec"] = -1

liste_date = df_train["datetime"].unique()

for i in range(len(liste_date)):
    if(i % 50 ==0):
        print(i, "/", len(liste_date), end="\r")
unique_date =liste_date[i]
for unique_date in df_train["datetime"].unique():
  # Récuperer le prix de l'electricite a ce jour la
  df_elec_filtered = df_electricity[df_electricity["origin_date"] == unique_date]
  if len(df_elec_filtered) != 0:
    if len(df_elec_filtered) == 1:
      elec_price = df_elec_filtered["euros_per_mwh"]
    else : 
      elec_price = df_elec_filtered.iloc[0]["euros_per_mwh"]

    # Pour toutes les lignes de df_train a ce jour la, insérer le prix
    df_train_filtered = df_train[df_train["datetime"] == unique_date]
    df_train.loc[df_train_filtered.index, "price_elec"] = elec_price.values[0]

C:\Users\Jawad\AppData\Local\Temp\ipykernel_14556\3953736527.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '199.99' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_train.loc[df_train_filtered.index, "price_elec"] = elec_price.values[0]


In [19]:
df_train.describe()

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec
count,100000.000000,100000.000000,100000.000000,99975.000000,100000.000000,100000.000000,1.000000e+05,100000.000000,100000.000000
mean,7.305100,0.535420,1.902090,274.941244,0.494670,322.720090,1.011847e+06,33.062190,156.808504
std,4.782535,0.498746,1.082199,911.820333,0.499974,182.181967,5.812419e+05,19.598742,118.848259
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.100000e+01,0.000000,-10.060000
25%,3.000000,0.000000,1.000000,0.328000,0.000000,167.000000,5.093938e+05,16.000000,84.760000
50%,7.000000,1.000000,2.000000,30.500000,0.000000,324.000000,1.013122e+06,33.000000,128.330000
75%,11.000000,1.000000,3.000000,179.827500,1.000000,479.000000,1.513979e+06,50.000000,199.960000
max,15.000000,1.000000,3.000000,15332.710000,1.000000,637.000000,2.018350e+06,68.000000,4000.000000
